In [ ]:
from qit.Qclient import Qclient
from qit.utils import data_utils
from qit.utils import preprocess
from sklearn.model_selection import train_test_split
from sklearn import metrics
import pandas as pd
import numpy as np
import json
import time
import subprocess
import warnings
warnings.filterwarnings('ignore')

In [ ]:
username = "your_qit_access_user_name"
secrect_key = 'your_qit_access_secrect_key'
client = Qclient(username,secrect_key)
datasets = ['bank', 'biomechanical']

# Prepare Data

In [ ]:
#You can use your own data, data provided by Qindom is just for demonstration
data_df = pd.DataFrame(columns = ['train_X' , 'valid_X', 'train_y' , 'valid_y','train_X_y'], index=datasets)
for dataset in datasets:
    print("start data preparation for dataset: " + dataset)
    
    train_valid_X, train_valid_y = data_utils.get_data(dataset)
    # Split the data for train and validation
    train_X, valid_X, train_y, valid_y = train_test_split(train_valid_X, train_valid_y, train_size=.7)
    train_X_y = np.concatenate((train_X, train_y.reshape(train_y.size,1)), axis=1)
    
    data_df['train_X'][dataset] = train_X
    data_df['train_y'][dataset] = train_y
    data_df['valid_X'][dataset] = valid_X
    data_df['valid_y'][dataset] = valid_y
    data_df['train_X_y'][dataset] = train_X_y
    
    np.savetxt("c:\\users\\fores\\Demo\\data\\" + dataset + "_train_X_y.csv", train_X_y, delimiter=',') #change to your own directory
    np.savetxt("c:\\users\\fores\\Demo\\data\\" + dataset + "_valid_X.csv", valid_X, delimiter=',') #change to your own directory
    for data in ['train_X_y','valid_X']:
        #change to your own directory
        cmdln = "C:\\Amazon\\AWSCLI\\bin\\aws.cmd s3 cp C:\\Users\\fores\\Demo\\data\\" + dataset + "_" + data + ".csv s3://qitdata/" + username + "/" + dataset + "_" + data + ".csv"
        p = subprocess.Popen(cmdln, shell=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT)
        for line in p.stdout.readlines():
            print(line)
            
    print("finished data preparation for dataset:" + dataset)
    
print("Finished all data preparation")

# Train & Predict

In [ ]:
clf_df = pd.DataFrame(columns = ['Qit'], index=datasets)
prd_df = pd.DataFrame(columns = ['Qit'], index=datasets)
for dataset in datasets:
    
    print("start to train model for: " + dataset)

    train_X = data_df['train_X'][dataset]
    train_y = data_df['train_y'][dataset]
    train_X_y = data_df['train_X_y'][dataset]
    
    model_id = client.train_model_s3(training_data=username +'/' + dataset + '_train_X_y.csv', clf_type='QBOOST_IT')
    clf_df['Qit'][dataset] = model_id['body']
    print("model id:" + str(clf_df['Qit'][dataset]))
    
    print("start predict for dataset: " + dataset)
    
    valid_X = data_df['valid_X'][dataset]
    valid_y = data_df['valid_y'][dataset]
    
#Get prediction result from Qit    
    prediction_result = client.predict_sync(test_data=username + '/' + dataset + '_valid_X.csv', modelId=clf_df['Qit'][dataset]) 
    while 'Models are not ready yet' in prediction_result:
        print(prediction_result + ", will try after a minute")
        time.sleep(60)
        prediction_result = client.predict_sync(test_data=username + '/' + dataset + '_valid_X.csv', modelId=clf_df['Qit'][dataset])
    prd_df['Qit'][dataset] = np.array(json.loads(prediction_result)['y_test_pred'])
    
#Calculate accuracy   
    qit_accuracy = metrics.accuracy_score(valid_y, prd_df['Qit'][dataset])
  
    print("finished predict for dataset: " + dataset + " with accuracy " + qit_accuracy)